# HW 11, Daniil Glukhovskiy, problem 2

## Data retrieval

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import qrpm_funcs as qf
from scipy.stats import skew, kurtosis
#Get 3 currencies until the end of
#previous year. Form sample covariance matrix
#and do simple efficient frontier calculations

lastday=qf.LastYearEnd()
#Swiss franc, pound sterling, Japanese Yen
seriesnames=['DEXSZUS','DEXUSUK','DEXJPUS']
cdates,ratematrix=qf.GetFREDMatrix(seriesnames,enddate=lastday)
multipliers=[-1,1,-1]

lgdates,difflgs=qf.levels_to_log_returns(cdates,ratematrix,multipliers)

#compute means, (co)variances, skewness and kurtosis
d=np.array(difflgs)
m=np.mean(d,axis=0)
c=np.cov(d.T)
ske = skew(d, axis = 0)
kurt = kurtosis(d, axis = 0)

#display the output
#vectors and matrices are in fractional units;
#    fraction*100=percent
#    fraction*10000=basis point
#    (fraction^2)*10000=percent^2
np.set_printoptions(precision=4)
print("From",lgdates[0],"to",lgdates[-1],"(",len(lgdates),"observations):")
print("\nMeans:",m*10000,"bps/day")
print("Skew:",ske)
print("kurtosis", kurt)
print("(CHF, GBP, JPY)\n")
print("  ",c[0]*10000)
print("C=",c[1]*10000,"    (4.20)")
print("  ",c[2]*10000)
print(f'(%/day)\N{SUPERSCRIPT TWO} units')

From 1971-01-05 to 2019-12-31 ( 12285 observations):

Means: [ 1.2022 -0.4963  0.9699] bps/day
Skew: [ 0.4007 -0.3845  0.6605]
kurtosis [14.2741  7.2467  9.522 ]
(CHF, GBP, JPY)

   [0.5113 0.2451 0.2175]
C= [0.2451 0.3551 0.1144]     (4.20)
   [0.2175 0.1144 0.4094]
(%/day)² units


## Empirical kurtosis of portfolio

In [29]:
#computing the kurtosis of the real returns of equally-distributed portfolio
w = np.ones((3))/3
real_returns = np.exp(d) - 1
ptf_returns = np.dot(real_returns, w)
print('Empirical kurtosis of equally weighted portfolio:', kurtosis(ptf_returns))

Empirical kurtosis of equally weighted portfolio: 4.742059984195001


## Fitting empirical kurtosis of each currency to mixture of normals

In [11]:
#plugging in kurtosis of each currency's log returns into the formula (*) from pdf file.
#take the negative sign for solution to exist. 
sigma_low = np.sqrt((38 - np.sqrt(38**2 - 4*19*(20 - kurt/3 - 1)))/38)
sigma_high = np.sqrt(20 - 19*sigma_low**2)
print("( CHF  ,  GBP  , JPY )")
print(sigma_low, ' - low part of standard deviation')
print(sigma_high, ' - high part of standard deviation')

( CHF  ,  GBP  , JPY )
[0.7068 0.8021 0.7689]  - low part of standard deviation
[3.2416 2.7883 2.9607]  - high part of standard deviation


## Running Monte-Carlo simulation

In [34]:
#Show the Cholesky decomposition
#of the CHF-GPB-JPY covariance matrix
chol=np.linalg.cholesky(c)
print('(Cholesky decomposition of 3-currency sample covariance matrix:\n',chol*100)

(Cholesky decomposition of 3-currency sample covariance matrix:
 [[0.715  0.     0.    ]
 [0.3428 0.4874 0.    ]
 [0.3042 0.0207 0.5626]]


In [40]:
from tabulate import tabulate
headers=['Statistic','Value']
count = 12285
#Generate random draws; use fixed seed to be replicable
seed=np.random.seed(12345678)
s_trial = np.zeros((count, 3), )

for trial in range(count):
    s_trial_1 = np.array([0.0, 0.0, 0.0], dtype='f8')
#Independently sample from mixtures of normals
    for i in range(3):
        p = np.random.uniform()
        if p < 0.95:
            s_trial_1[i]=np.random.normal(0, sigma_low[i])
        else:
            s_trial_1[i]=np.random.normal(0, sigma_high[i])
    s_trial[trial,:] = s_trial_1

logr_trial = np.matmul(chol, s_trial.T).T
#logr_trial has Monte Carlo log-returns; transform to returns
r_trial = np.expm1(logr_trial)
#Get trial portfolio returns
r_ptrial=np.matmul(r_trial, w)

statnames,mettrial,tabtrial=qf.StatsTable(r_ptrial)
print(tabulate(tabtrial, headers, tablefmt='fancy_grid'))

╒════════════════════╤═══════════════╕
│ Statistic          │         Value │
╞════════════════════╪═══════════════╡
│ Count              │ 12285         │
├────────────────────┼───────────────┤
│ Min                │    -0.0365036 │
├────────────────────┼───────────────┤
│ Max                │     0.041665  │
├────────────────────┼───────────────┤
│ Mean               │     1.22e-05  │
├────────────────────┼───────────────┤
│ Median             │    -1.77e-05  │
├────────────────────┼───────────────┤
│ Standard Deviation │     0.0051426 │
├────────────────────┼───────────────┤
│ Skewness           │     0.169091  │
├────────────────────┼───────────────┤
│ Excess Kurtosis    │     7.25797   │
├────────────────────┼───────────────┤
│ Jarque-Bera        │ 27023.2       │
├────────────────────┼───────────────┤
│ Chi-Squared p      │     0         │
├────────────────────┼───────────────┤
│ Serial Correlation │     0.0054404 │
├────────────────────┼───────────────┤
│ 99% VaR            │   

Empirical kurtosis of portfolio returns is lower then the simulated kurtosis.